# Divide and Conquer Algorithm
<p>
    Optimal Algorithm
</p>

In [1]:
import os
import sys
import time
import datetime
import math
import pandas as pd

### Read data from file

In [2]:
allStarbucksLocations = pd.read_excel(open('./data/Data.xlsx','rb'),sheet_name='Sheet1')

### Data Cleanup

##### Use to get list of columns from data frame
```python
list(allStarbucksLocations.columns)
```

In [3]:
allStarbucksLocations = allStarbucksLocations[allStarbucksLocations["Brand"] == "Starbucks"]
coordinates = allStarbucksLocations[["Store Number","Longitude","Latitude"]]
coordinates.shape

(17774, 3)

#### Drop Null Longitude & Latitude Values

In [4]:
coordinates = coordinates.dropna(subset=["Longitude","Latitude"])
coordinates.shape

(17774, 3)

#### Drop Duplicate Longitude & Latitude Values

In [5]:
coordinates.drop_duplicates(subset=["Longitude","Latitude"],keep="first",inplace=True)
coordinates.shape

(17774, 3)

#### Sort DataFrame on Longitude

In [6]:
sortedCoordinates = coordinates.sort_values(by=['Longitude'])

### Convert dataframe to tuple

In [7]:
coordinatesTuple = [tuple(x) for x in sortedCoordinates.values]

### Split into two sets

In [8]:
P1 = round(len(coordinatesTuple)/2)
coordinatesTuple_Set1 = coordinatesTuple[:P1]

In [9]:
P2 = len(coordinatesTuple)
coordinatesTuple_Set2 = coordinatesTuple[P1:P2]

### Define a function that returns a set of store pairs

In [10]:
def findUniqueStores(pairs):
    listOfPairs = []
    
    for c1 in range(len(pairs)):
        for c2 in range(len(pairs)):
            if ( c1 == c2 ):
                pass
            else:
                orderedPair = ( c1 , c2 )                
                listOfPairs.append(orderedPair)
    return listOfPairs

### Define function: euclidean_distance

In [11]:
def euclidean_distance(value1, value2):
    distLongitude = (( value1[1] - value2[1] )**2)
    distLatitude = (( value1[2] - value2[2] )**2)
    
    return ( value1[0] , value2[0] , distLongitude , distLatitude )

### Combine the two functions to get euclidean distance of unique stores

In [12]:
def findMinDistance(data):
    global minDistance
    global newTuple
    newTuple = []
    iteration = 0
    for s in findUniqueStores(data):
        
        d = euclidean_distance( data[s[0]] , data[s[1]] )
        
        if iteration == 0:
            minDist = math.sqrt( d[2] + d[3] )
            minDistance = d
            
            newTuple.append((iteration,d[0],d[1],d[2],d[3],minDist))
            
        else:
            dist = math.sqrt( d[2] + d[3] )
            
            newTuple.append((iteration,d[0],d[1],d[2],d[3],minDist))
            
            if ( dist <= minDist ):
                minDist = dist
                minDistance = d
            else:
                pass
        
        iteration += 1
        
    print("Number of Inputs: {0}".format(len(data)))
    print("Number of Comparisons: {0}".format(iteration))
    print("Closest Starbucks Stores: Store Numbers {0} and {1}".format(minDistance[0],minDistance[1]))
    print("Longitude Distance: {:.5f}, Latitude Distance: {:.5f}".format(minDistance[2],minDistance[3]))
    return minDistance

### Set 1 Data

In [13]:
start_time = time.time()
minDistance_Set1 = findMinDistance(coordinatesTuple_Set1)
distance_Set1 = math.sqrt( minDistance_Set1[2] + minDistance_Set1[3] )
newTuple_Set1 = newTuple
end_time = time.time()
timeToComputeSet1 = datetime.timedelta(seconds=(end_time - start_time))
print("Time taken:{0}".format(timeToComputeSet1))

Number of Inputs: 8887
Number of Comparisons: 78969882
Closest Starbucks Stores: Store Numbers 8485-57681 and 8428-28454
Longitude Distance: 0.00010, Latitude Distance: 0.00000
Time taken:0:01:22.370090


### Set 2 Data

In [14]:
start_time = time.time()
minDistance_Set2 = findMinDistance(coordinatesTuple_Set2)
distance_Set2 = math.sqrt( minDistance_Set2[2] + minDistance_Set2[3] )
newTuple_Set2 = newTuple
end_time = time.time()
timeToComputeSet2 = datetime.timedelta(seconds=(end_time - start_time))
print("Time taken:{0}".format(timeToComputeSet2))

Number of Inputs: 8887
Number of Comparisons: 78969882
Closest Starbucks Stores: Store Numbers 25282-240406 and 25278-240393
Longitude Distance: 0.00010, Latitude Distance: 0.00000
Time taken:0:01:21.893708


### Compare the two sets of data
Set the smallest euclidean distance as the mininum distance

In [15]:
start_time = time.time()
if distance_Set1 < distance_Set2:
    dataSummary = newTuple_Set1
    mindis = distance_Set1
    closestStores = minDistance_Set1
    
    print("Store Numbers {0} and {1}".format(minDistance_Set1[0],minDistance_Set1[1]))
    print("Longitude Distance: {:.5f}".format(minDistance_Set1[2]))
    print("Latitude Distance: {:.5f}".format(minDistance_Set1[3]))

else:
    dataSummary = newTuple_Set2
    mindis = distance_Set2
    closestStores = minDistance_Set2
    
    print("Store Numbers {0} and {1}".format(minDistance_Set2[0],minDistance_Set2[1]))
    print("Longitude Distance:{:.5f}".format(minDistance_Set2[2]))
    print("Latitude Distance: {:.5f}".format(minDistance_Set2[3]))
    
end_time = time.time()
timeToCompareDatasets = datetime.timedelta(seconds=(end_time - start_time))
cumulativeTime = timeToComputeSet1 + timeToComputeSet2 + timeToCompareDatasets
print("Total Time:{0}".format(cumulativeTime))

Store Numbers 25282-240406 and 25278-240393
Longitude Distance:0.00010
Latitude Distance: 0.00000
Total Time:0:02:44.264298


### Determine the upper and lower bounds

In [16]:
start_time = time.time()
LongitudeMid = coordinatesTuple[P1][1]
print("The minimum distance recorded is: {0}".format(mindis))
print("The longitudnal midpoint is: {:.7f}".format(LongitudeMid))

The minimum distance recorded is: 0.009999999999990905
The longitudnal midpoint is: -82.4900000


In [17]:
for i in range(P1):
    j = P1 - i    
    if LongitudeMid - coordinatesTuple[j][1] < mindis:
        pass
    else:
        Lowerbound = j
        return_i = P1 - 1
        break

In [18]:
for i in range(P1,P2):
    if coordinatesTuple[i][1] - LongitudeMid < mindis:
        pass
    else:
        Upperbound = i
        break

In [19]:
end_time = time.time()
timeToDetermineUpperLowerBound = datetime.timedelta(seconds=(end_time - start_time))
cumulativeTime = timeToComputeSet1 + timeToComputeSet2 + timeToCompareDatasets + timeToDetermineUpperLowerBound
print("Lowerbound value is: {0}".format(Lowerbound))
print("Upperbound value is: {0}".format(Upperbound))
print("Total Time:{0}".format(cumulativeTime))

Lowerbound value is: 8886
Upperbound value is: 8892
Total Time:0:02:44.311335


### Find shortest distance in new array

In [34]:
start_time = time.time()
for i in range(Lowerbound,Upperbound+1):
    Lng1 = coordinatesTuple[i][1]
    Lat1 = coordinatesTuple[i][2]
    
    for j in range(i + 1, Upperbound + 1):        
        Lng2 = coordinatesTuple[j][1]
        Lat2 = coordinatesTuple[j][2]
        
        dis = math.sqrt(((Lng1 - Lng2)**2) + ((Lat1 - Lat2)**2))
                
        if dis <= mindis:
            mindis = dis            
            Store1 = coordinatesTuple[i]
            Store2 = coordinatesTuple[j]
        else:
            pass
end_time = time.time()
timeToDetermineShortestDistance = datetime.timedelta(seconds=(end_time - start_time))
cumulativeTime = timeToComputeSet1 + \
                 timeToComputeSet2 + \
                 timeToCompareDatasets +\
                 timeToDetermineUpperLowerBound + \
                 timeToDetermineShortestDistance

try: Store1
except NameError: Store1 = closestStores[0]

try: Store2
except NameError: Store2 = closestStores[1]

print("Total Time:{0}".format(cumulativeTime))
print("The Closest Starbucks stores are {0} and {1}".format(Store1,Store2))

Total Time:0:02:44.311335
The Closest Starbucks stores are 25282-240406 and 25278-240393
